In [2]:
# Импортируем необходимые библиотеки для предобработки датасетов

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import ExtraTreeRegressor
from tqdm import tqdm
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression

In [3]:
import warnings

warnings.filterwarnings('ignore')

In [4]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [5]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [6]:
bgg_data = pd.read_csv('data/bgg_data_full_eda.csv')

In [7]:
bgg_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168509 entries, 0 to 168508
Data columns (total 69 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   name                  168509 non-null  object 
 1   description           168509 non-null  object 
 2   rating                168509 non-null  float64
 3   bayes_rating          168509 non-null  float64
 4   usersrated            168509 non-null  int64  
 5   bggrank               168509 non-null  int64  
 6   stddev                168509 non-null  float64
 7   owned                 168509 non-null  int64  
 8   trading               168509 non-null  int64  
 9   wanting               168509 non-null  int64  
 10  wishing               168509 non-null  int64  
 11  numweights            168509 non-null  int64  
 12  averageweight         168509 non-null  float64
 13  yearpublished         168509 non-null  int64  
 14  minplayers            168509 non-null  int64  
 15  

Из данных полученых в EDA, о значимости признаков оставляем статистически значимые признаки

In [8]:
cols_del = ['name', 'description', 'rating',
            'playtime', 'maxplayers', 'minplaytime',
            'len_name', 'minplayers', 'len_description',
            'yearpublished', 'log_len_description', 'log_len_name',
            'log_minplaytime', 'log_playtime']

bgg_df = bgg_data.drop(cols_del, axis=1)

In [9]:
cols_dubls = ['bayes_rating', 'usersrated', 'bggrank',
            'stddev', 'owned', 'trading',
            'wanting', 'wishing', 'numweights',
            'averageweight', 'age']

bgg_df.drop(cols_dubls, axis=1, inplace=True)

Подбор модели машинного обучения

In [10]:
y = bgg_df['log_rating']

x = bgg_df.drop('log_rating', axis=1)

In [11]:
# Создадим тренировочную и валидационную выборки
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)

In [12]:
# Функции расчета ошибки
def MAPE(y_true, y_pred): 
    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    return np.mean(np.abs((y_true - y_pred) / y_true) * 100)


def MAE(y_true, y_pred):
    
    abs_error = np.abs(y_pred - y_true)
    sum_abs_error = np.sum(abs_error)
    
    return sum_abs_error / y_true.size

1. Обучим простую модель - Линейную регрессию

In [13]:
lr_model = LinearRegression()
lr_model.fit(x_train, y_train)

lr_predict = lr_model.predict(x_val)
# оцениваем точность
print(f"Точность модели по метрике MAPE: {(MAPE(y_val, lr_predict))*100:0.f}%")
print(f"Точность модели по метрике MAE: {(MAE(y_val, lr_predict))*100:0.2f}%")

Точность модели по метрике MAPE: inf%
Точность модели по метрике MAE: 27.29%


In [14]:
estimate = -1 * cross_val_score(lr_model,
                           x_train,
                           y_train,
                           cv=10,
                           scoring='neg_mean_absolute_error')

print(f"Точность модели по метрике MAE: {estimate.mean() * 100:0.2f}%")

Точность модели по метрике MAE: 27.41%


2. RandomForestRegressor

GridSearchCV на моей машине не отрабатывает, уходит в вечрный поиск. Хотя один проход около 3 минут.

In [ ]:
'''param_rfr_grid = [
{
    'n_estimators': range(100, 500, 100),
    'max_features': ['auto', 'sqrt', 'log2']
},
{
    'bootstrap': [False],
    'n_estimators': range(100, 500, 100),
    'max_features': ['auto', 'sqrt', 'log2']
},
]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_rfr_grid, cv=5,
                          scoring='neg_mean_absolute_error')
 
grid_search.fit(x_train.values,np.ravel(y_train.values))'''

In [ ]:
'''print(grid_search.best_params_)
print(f"Точность модели по метрике MAE: {grid_search.best_score_:0.2f}%")'''

In [15]:
rfr_model = RandomForestRegressor(n_estimators=500,
                                  max_features='auto',
                                  bootstrap=True,
                                  random_state = RANDOM_SEED,
                                  n_jobs = -1,
                                  verbose = 2,
                                  min_samples_split = 5,
                                  min_samples_leaf = 2)

rfr_model.fit(x_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 500
building tree 2 of 500building tree 3 of 500

building tree 4 of 500
building tree 5 of 500
building tree 6 of 500building tree 7 of 500

building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500building tree 34 of 500

building tree 35 of 500building tree 36 of 500

building tree 37 of 500
building tree 38 of 500


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    9.5s


building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
building tree 43 of 500
building tree 44 of 500
building tree 45 of 500
building tree 46 of 500
building tree 47 of 500
building tree 48 of 500
building tree 49 of 500
building tree 50 of 500
building tree 51 of 500
building tree 52 of 500
building tree 53 of 500
building tree 54 of 500
building tree 55 of 500
building tree 56 of 500
building tree 57 of 500
building tree 58 of 500
building tree 59 of 500
building tree 60 of 500
building tree 61 of 500
building tree 62 of 500
building tree 63 of 500
building tree 64 of 500
building tree 65 of 500
building tree 66 of 500
building tree 67 of 500
building tree 68 of 500
building tree 69 of 500building tree 70 of 500

building tree 71 of 500
building tree 72 of 500
building tree 73 of 500
building tree 74 of 500
building tree 75 of 500
building tree 76 of 500
building tree 77 of 500
building tree 78 of 500
building tree 79 of 500
building tree 80

[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   45.4s


building tree 155 of 500
building tree 156 of 500
building tree 157 of 500
building tree 158 of 500
building tree 159 of 500
building tree 160 of 500
building tree 161 of 500
building tree 162 of 500
building tree 163 of 500
building tree 164 of 500
building tree 165 of 500
building tree 166 of 500
building tree 167 of 500
building tree 168 of 500
building tree 169 of 500
building tree 170 of 500
building tree 171 of 500
building tree 172 of 500
building tree 173 of 500
building tree 174 of 500
building tree 175 of 500
building tree 176 of 500
building tree 177 of 500
building tree 178 of 500
building tree 179 of 500
building tree 180 of 500
building tree 181 of 500
building tree 182 of 500
building tree 183 of 500
building tree 184 of 500
building tree 185 of 500
building tree 186 of 500
building tree 187 of 500
building tree 188 of 500
building tree 189 of 500
building tree 190 of 500
building tree 191 of 500
building tree 192 of 500
building tree 193 of 500
building tree 194 of 500


[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.8min


building tree 357 of 500
building tree 358 of 500
building tree 359 of 500
building tree 360 of 500
building tree 361 of 500
building tree 362 of 500
building tree 363 of 500
building tree 364 of 500
building tree 365 of 500
building tree 366 of 500
building tree 367 of 500
building tree 368 of 500
building tree 369 of 500
building tree 370 of 500
building tree 371 of 500
building tree 372 of 500
building tree 373 of 500
building tree 374 of 500
building tree 375 of 500
building tree 376 of 500
building tree 377 of 500
building tree 378 of 500
building tree 379 of 500
building tree 380 of 500
building tree 381 of 500
building tree 382 of 500
building tree 383 of 500
building tree 384 of 500
building tree 385 of 500
building tree 386 of 500
building tree 387 of 500
building tree 388 of 500
building tree 389 of 500
building tree 390 of 500
building tree 391 of 500
building tree 392 of 500
building tree 393 of 500
building tree 394 of 500
building tree 395 of 500
building tree 396 of 500


[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.5min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 349 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    1.7s finished


NameError: name 'mape' is not defined

In [26]:
rfr_predict = rfr_model.predict(x_val)

# оцениваем точность
print(f"Точность модели по метрике MAPE: {(MAPE(y_val, rfr_predict))*100:0.2f}%")
print(f"Точность модели по метрике MAE: {(MAE(y_val, rfr_predict))*100:0.2f}%")

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 349 tasks      | elapsed:    0.9s


Точность модели по метрике MAPE: nan%
Точность модели по метрике MAE: 10.75%


[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    1.3s finished


3. CatBoost

Так же не отрабатывает.

In [ ]:
'''param_cb_grid = {
    'ITERATIONS': range(1000, 6000, 1000),
    'l2_leaf_reg': range(1, 6, 1),
    'bootstrap_type': ['Bayesian', 'Bernoulli', 'MVS'],
    'random_seed': RANDOM_SEED,
    'eval_metric': 'MAE',
}

cat_boost = CatBoostRegressor()
grid_search_2 = GridSearchCV(cbr_model, cbr_model, cv=5,
                          scoring='neg_mean_absolute_error')
 
grid_search_2.fit(x_train.values,np.ravel(y_train.values))'''

In [ ]:
'''print(grid_search_2.best_params_)
print(f"Точность модели по метрике MAE: {grid_search_2.best_score_:0.2f}%")'''

In [17]:
cbr_model = CatBoostRegressor(iterations = 6000,
                               learning_rate = 0.1,
                               l2_leaf_reg = 6,
                               random_seed = RANDOM_SEED,
                               eval_metric='MAE',
                               bootstrap_type = 'MVS')

cbr_model.fit(x_train, y_train,
               eval_set=(x_val, y_val),
               verbose_eval=100,
               use_best_model=True,
               plot=True)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.5380858	test: 0.5352972	best: 0.5352972 (0)	total: 197ms	remaining: 19m 41s
100:	learn: 0.1167808	test: 0.1172919	best: 0.1172919 (100)	total: 3.15s	remaining: 3m 3s
200:	learn: 0.1134835	test: 0.1143664	best: 0.1143552 (199)	total: 5.46s	remaining: 2m 37s
300:	learn: 0.1116319	test: 0.1128385	best: 0.1128377 (299)	total: 7.87s	remaining: 2m 29s
400:	learn: 0.1104152	test: 0.1119082	best: 0.1119046 (398)	total: 10.2s	remaining: 2m 22s
500:	learn: 0.1096681	test: 0.1113954	best: 0.1113954 (500)	total: 12.5s	remaining: 2m 17s
600:	learn: 0.1091448	test: 0.1110687	best: 0.1110687 (600)	total: 15.1s	remaining: 2m 15s
700:	learn: 0.1086591	test: 0.1108035	best: 0.1108005 (699)	total: 17.4s	remaining: 2m 11s
800:	learn: 0.1081189	test: 0.1104888	best: 0.1104762 (799)	total: 19.7s	remaining: 2m 8s
900:	learn: 0.1077445	test: 0.1102718	best: 0.1102718 (900)	total: 22s	remaining: 2m 4s
1000:	learn: 0.1073678	test: 0.1101085	best: 0.1101085 (1000)	total: 24.2s	remaining: 2m
1100:	lea

In [27]:
cbr_predict = cbr_model.predict(x_val)

# оцениваем точность
print(f"Точность модели по метрике MAPE: {(MAPE(y_val, cbr_predict))*100:0.2f}%")
print(f"Точность модели по метрике MAE: {(MAE(y_val, cbr_predict))*100:0.2f}%")

Точность модели по метрике MAPE: inf%
Точность модели по метрике MAE: 10.92%


4. BaggingRegressor с RandomForestRegressor

In [20]:
br_rfr_model = BaggingRegressor(rfr_model, n_estimators=3, n_jobs=-1, random_state=RANDOM_SEED)
br_rfr_model.fit(x_train, y_train)

BaggingRegressor(base_estimator=RandomForestRegressor(min_samples_leaf=2,
                                                      min_samples_split=5,
                                                      n_estimators=500,
                                                      n_jobs=-1,
                                                      random_state=42,
                                                      verbose=2),
                 n_estimators=3, n_jobs=-1, random_state=42)

In [28]:
br_rfr_predict = br_rfr_model.predict(x_val)
# оцениваем точность
print(f"Точность модели по метрике MAPE: {(MAPE(y_val, br_rfr_predict))*100:0.2f}%")
print(f"Точность модели по метрике MAE: {(MAE(y_val, br_rfr_predict))*100:0.2f}%")

Точность модели по метрике MAPE: nan%
Точность модели по метрике MAE: 10.77%


5. GradientBoostingRegressor

In [22]:
gbr_model = GradientBoostingRegressor(random_state=RANDOM_SEED, n_estimators=200)
gbr_model.fit(x_train, y_train)

GradientBoostingRegressor(n_estimators=200, random_state=42)

In [29]:
gbr_predict = gbr_model.predict(x_val)

# оцениваем точность
print(f"Точность модели по метрике MAPE: {(MAPE(y_val, gbr_predict))*100:0.2f}%")
print(f"Точность модели по метрике MAE: {(MAE(y_val, gbr_predict))*100:0.2f}%")

Точность модели по метрике MAPE: inf%
Точность модели по метрике MAE: 11.86%


6. Stacking with RandomForestRegressor and CatBoostRegressor

In [24]:
st_models = [cbr_model, rfr_model]

def stacking_model_predict(d_models, d_X_train, d_y_train, d_X_val):
    d_df = pd.DataFrame()
    for model_ in tqdm(d_models):
        model_.fit(d_X_train, d_y_train)
        y_pred = model_.predict(d_X_val)
        d_df[str(model_)[:6]] = y_pred               
    return d_df

temp_df = stacking_model_predict(st_models, x_train, y_train, x_val)
temp_df['y_pred'] = temp_df.mean(axis=1)

  0%|          | 0/2 [00:00<?, ?it/s]

0:	learn: 0.5380858	total: 78.9ms	remaining: 7m 53s
1:	learn: 0.4939154	total: 96.2ms	remaining: 4m 48s
2:	learn: 0.4537596	total: 121ms	remaining: 4m 2s
3:	learn: 0.4180159	total: 144ms	remaining: 3m 36s
4:	learn: 0.3860797	total: 170ms	remaining: 3m 23s
5:	learn: 0.3572371	total: 194ms	remaining: 3m 14s
6:	learn: 0.3317959	total: 220ms	remaining: 3m 8s
7:	learn: 0.3091099	total: 247ms	remaining: 3m 5s
8:	learn: 0.2887891	total: 274ms	remaining: 3m 2s
9:	learn: 0.2708939	total: 317ms	remaining: 3m 9s
10:	learn: 0.2547676	total: 359ms	remaining: 3m 15s
11:	learn: 0.2404746	total: 394ms	remaining: 3m 16s
12:	learn: 0.2278893	total: 422ms	remaining: 3m 14s
13:	learn: 0.2166627	total: 449ms	remaining: 3m 12s
14:	learn: 0.2066551	total: 488ms	remaining: 3m 14s
15:	learn: 0.1976004	total: 523ms	remaining: 3m 15s
16:	learn: 0.1894238	total: 566ms	remaining: 3m 19s
17:	learn: 0.1822016	total: 606ms	remaining: 3m 21s
18:	learn: 0.1758183	total: 645ms	remaining: 3m 22s
19:	learn: 0.1702000	tota

160:	learn: 0.1144607	total: 4.32s	remaining: 2m 36s
161:	learn: 0.1144091	total: 4.34s	remaining: 2m 36s
162:	learn: 0.1143846	total: 4.37s	remaining: 2m 36s
163:	learn: 0.1143832	total: 4.39s	remaining: 2m 36s
164:	learn: 0.1143213	total: 4.41s	remaining: 2m 35s
165:	learn: 0.1143203	total: 4.43s	remaining: 2m 35s
166:	learn: 0.1142948	total: 4.46s	remaining: 2m 35s
167:	learn: 0.1142754	total: 4.48s	remaining: 2m 35s
168:	learn: 0.1142395	total: 4.5s	remaining: 2m 35s
169:	learn: 0.1141837	total: 4.52s	remaining: 2m 35s
170:	learn: 0.1141721	total: 4.54s	remaining: 2m 34s
171:	learn: 0.1141509	total: 4.57s	remaining: 2m 34s
172:	learn: 0.1141185	total: 4.59s	remaining: 2m 34s
173:	learn: 0.1140760	total: 4.62s	remaining: 2m 34s
174:	learn: 0.1140260	total: 4.64s	remaining: 2m 34s
175:	learn: 0.1140304	total: 4.66s	remaining: 2m 34s
176:	learn: 0.1139910	total: 4.68s	remaining: 2m 34s
177:	learn: 0.1139582	total: 4.71s	remaining: 2m 33s
178:	learn: 0.1139248	total: 4.73s	remaining: 2

316:	learn: 0.1113759	total: 7.78s	remaining: 2m 19s
317:	learn: 0.1113599	total: 7.8s	remaining: 2m 19s
318:	learn: 0.1113536	total: 7.83s	remaining: 2m 19s
319:	learn: 0.1113396	total: 7.85s	remaining: 2m 19s
320:	learn: 0.1113420	total: 7.87s	remaining: 2m 19s
321:	learn: 0.1113386	total: 7.9s	remaining: 2m 19s
322:	learn: 0.1113329	total: 7.92s	remaining: 2m 19s
323:	learn: 0.1113085	total: 7.94s	remaining: 2m 19s
324:	learn: 0.1112860	total: 7.96s	remaining: 2m 19s
325:	learn: 0.1112592	total: 7.99s	remaining: 2m 19s
326:	learn: 0.1112421	total: 8.01s	remaining: 2m 19s
327:	learn: 0.1112284	total: 8.04s	remaining: 2m 18s
328:	learn: 0.1112143	total: 8.06s	remaining: 2m 18s
329:	learn: 0.1112081	total: 8.08s	remaining: 2m 18s
330:	learn: 0.1112003	total: 8.1s	remaining: 2m 18s
331:	learn: 0.1111820	total: 8.13s	remaining: 2m 18s
332:	learn: 0.1111631	total: 8.15s	remaining: 2m 18s
333:	learn: 0.1111613	total: 8.17s	remaining: 2m 18s
334:	learn: 0.1111367	total: 8.19s	remaining: 2m 

473:	learn: 0.1098201	total: 11.4s	remaining: 2m 12s
474:	learn: 0.1098196	total: 11.4s	remaining: 2m 12s
475:	learn: 0.1098191	total: 11.4s	remaining: 2m 12s
476:	learn: 0.1098218	total: 11.4s	remaining: 2m 12s
477:	learn: 0.1098130	total: 11.4s	remaining: 2m 12s
478:	learn: 0.1098026	total: 11.5s	remaining: 2m 12s
479:	learn: 0.1097912	total: 11.5s	remaining: 2m 12s
480:	learn: 0.1097879	total: 11.5s	remaining: 2m 12s
481:	learn: 0.1097872	total: 11.5s	remaining: 2m 12s
482:	learn: 0.1097826	total: 11.6s	remaining: 2m 12s
483:	learn: 0.1097726	total: 11.6s	remaining: 2m 12s
484:	learn: 0.1097702	total: 11.6s	remaining: 2m 11s
485:	learn: 0.1097682	total: 11.6s	remaining: 2m 11s
486:	learn: 0.1097680	total: 11.6s	remaining: 2m 11s
487:	learn: 0.1097601	total: 11.7s	remaining: 2m 11s
488:	learn: 0.1097599	total: 11.7s	remaining: 2m 11s
489:	learn: 0.1097570	total: 11.7s	remaining: 2m 11s
490:	learn: 0.1097567	total: 11.7s	remaining: 2m 11s
491:	learn: 0.1097486	total: 11.7s	remaining: 

...

[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   42.4s


building tree 156 of 500
building tree 157 of 500
building tree 158 of 500
building tree 159 of 500
building tree 160 of 500
building tree 161 of 500
building tree 162 of 500
building tree 163 of 500
building tree 164 of 500
building tree 165 of 500
building tree 166 of 500
building tree 167 of 500
building tree 168 of 500
building tree 169 of 500
building tree 170 of 500
building tree 171 of 500
building tree 172 of 500
building tree 173 of 500
building tree 174 of 500
building tree 175 of 500
building tree 176 of 500
building tree 177 of 500
building tree 178 of 500
building tree 179 of 500
building tree 180 of 500
building tree 181 of 500
building tree 182 of 500
building tree 183 of 500
building tree 184 of 500
building tree 185 of 500
building tree 186 of 500
building tree 187 of 500
building tree 188 of 500
building tree 189 of 500
building tree 190 of 500
building tree 191 of 500
building tree 192 of 500
building tree 193 of 500
building tree 194 of 500
building tree 195 of 500


[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.6min


building tree 358 of 500
building tree 359 of 500
building tree 360 of 500
building tree 361 of 500
building tree 362 of 500
building tree 363 of 500
building tree 364 of 500
building tree 365 of 500
building tree 366 of 500
building tree 367 of 500
building tree 368 of 500
building tree 369 of 500
building tree 370 of 500
building tree 371 of 500
building tree 372 of 500
building tree 373 of 500
building tree 374 of 500
building tree 375 of 500
building tree 376 of 500
building tree 377 of 500
building tree 378 of 500
building tree 379 of 500
building tree 380 of 500
building tree 381 of 500
building tree 382 of 500
building tree 383 of 500
building tree 384 of 500building tree 385 of 500

building tree 386 of 500
building tree 387 of 500
building tree 388 of 500
building tree 389 of 500
building tree 390 of 500
building tree 391 of 500
building tree 392 of 500
building tree 393 of 500
building tree 394 of 500
building tree 395 of 500
building tree 396 of 500
building tree 397 of 500


[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.3min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 349 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    1.3s finished
100%|██████████| 2/2 [04:42<00:00, 141.50s/it]


In [30]:
st_predict = np.array(temp_df['y_pred'])
# оцениваем точность
print(f"Точность модели по метрике MAPE: {(MAPE(y_val, st_predict))*100:0.2f}%")
print(f"Точность модели по метрике MAE: {(MAE(y_val, st_predict))*100:0.2f}%")

Точность модели по метрике MAPE: inf%
Точность модели по метрике MAE: 10.69%
